In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from datetime import datetime
import psycopg2
import pyodbc

In [ ]:
engine = create_engine('sqlite:///chinook.db')


In [ ]:
engine = create_engine('sqlite:///chinook.db')
customers = pd.read_sql_query("""SELECT FirstName, LastName, Phone, Fax, Email FROM customers; """, con=engine.connect())
artists = pd.read_sql_query("""SELECT Name FROM artists;""", con=engine.connect())
invoice_items = pd.read_sql_query("""SELECT UnitPrice, Quantity FROM invoice_items;""", con=engine.connect())
location = pd.read_sql_query("""SELECT BillingAddress as Address, BillingCity as City, BillingState as State, BillingCountry as Country,BillingPostalCode as PostalCode from invoices;""", con=engine.connect())
employees = pd.read_sql_query("""SELECT FirstName, LastName, Title, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email From employees;""", con=engine.connect())
albums = pd.read_sql_query("""SELECT Title FROM albums;""", con=engine.connect()) 
tracks = pd.read_sql_query("""SELECT t.Name, mt.Name AS MediaType, g.Name AS Genre, t.Composer, t.Milliseconds, t.Bytes, t.UnitPrice 
       FROM tracks t
       INNER JOIN
       [genres] g ON t.GenreId = g.GenreId
       INNER JOIN
       media_types mt ON t.MediaTypeId = mt.MediaTypeId """, con=engine.connect()) 
fs=pd.read_sql_query("""SELECT invoice_items.InvoiceLineId as InvoicesID, customers.CustomerId AS CustomerID, employees.EmployeeId AS EmployeeID, dim_time.timeID AS TimeID,
invoices.InvoiceId as LocationID, tracks.TrackId AS TrackId, artists.ArtistId AS ArtistId, albums.AlbumId AS AlbumId, invoices.Total AS Total
FROM invoices INNER JOIN customers
ON invoices.CustomerId= customers.CustomerId
INNER JOIN invoice_items
ON invoice_items.InvoiceId= invoices.InvoiceId
INNER JOIN tracks
ON tracks.TrackId=  invoice_items.TrackId
INNER JOIN albums
ON tracks.AlbumId= albums.AlbumId
INNER JOIN artists 
ON albums.ArtistId= artists.ArtistId 
INNER JOIN Dim_Time
ON invoices.InvoiceDate= substr(dim_time.date,0,20)
INNER JOIN employees
ON employees.EmployeeId= customers.SupportRepId;""", con=engine.connect())

In [ ]:
fecha_I='2009-01-01'
fecha_F='2030-12-31'
date = pd.DataFrame({'date': pd.date_range(fecha_I, fecha_F)})
date['año'] = date.date.dt.year
date['mes'] = date.date.dt.month
date['dia'] = date.date.dt.day
date['nombre_dia'] = date.date.dt.day_name()
date['dia_semana'] = date.date.dt.dayofweek
date['semana'] = date.date.dt.weekofyear
date['trimestre'] = date.date.dt.quarter
date = dfdate[['date', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']]
date.to_sql(name='Dim_Time', con=engine.connect(), if_exists='append',index=False)

In [ ]:
conn = sqlalchemy.create_engine('mssql+pyodbc://localhost/DW_Sales_Music?driver=SQL+Server+Native+Client+11.0')

In [ ]:
albums.to_sql(name='Dim_Album', con=conn, if_exists='append',index=False)
artists.to_sql(name='Dim_Artist', con=conn, if_exists='append',index=False)
customers.to_sql(name='Dim_Customer', con=conn, if_exists='append',index=False)  
invoice_items.to_sql(name='Dim_Invoice_Item', con=conn, if_exists='append',index=False)
location.to_sql(name='Dim_Location', con=conn, if_exists='append',index=False)
date.to_sql(name='Dim_Time', con=conn, if_exists='append',index=False)
tracks.to_sql(name='Dim_Track', con=conn, if_exists='append',index=False)
employees.to_sql(name='Dim_Employee', con=conn, if_exists='append',index=False)
fs.to_sql(name='Fact_Sales', con=conn, if_exists='append',index=False)